## GridWorld
Ph.D Leonarod A, Espinosa, M.Sc Andrej Scherbakov-Parland, BIT Kristoffer Kuvaja Adolfsson

### Bibliography:

* Sutton, Richard S., and Andrew G. Barto. Reinforcement learning: An introduction. MIT press, 2018.
http://incompleteideas.net/book/bookdraft2017nov5.pdf  (chapter 4)

In [1]:
# imports
import numpy as np
from IPython.display import clear_output
import time

In [10]:
class GridWorld(object):
    """ Gridworld defined by m x n matrix with
    terminal states at top left corner and bottom right corner.
    State transitions are deterministic; attempting to move
    off the grid leaves the state unchanged, and rewards are -1 on
    each step. 

    In this implementation we model the environment like a game
    where an agent moves around.
    """
    def __init__(self, m, n):
        self.m = m
        self.n = n
        self.grid = np.zeros((m,n))
        self.stateSpace = [i+1 for i in range(self.m*self.n-2)]
        self.stateSpacePlus = [i for i in range(self.m*self.n)]        
        self.actionSpace = {'up': -self.m, 'down': self.m, 
                            'left': -1, 'right': 1}
        self.p = self.initP() # probability functions
        self.agentPosition = np.random.choice(self.stateSpace)
        x, y = self.getAgentRowAndColumn() 
        self.grid[x][y] = 1

    def getAgentRowAndColumn(self):
        x = self.agentPosition // self.m
        y = self.agentPosition % self.n
        return x, y

    def setState(self, state):
        x, y = self.getAgentRowAndColumn() 
        self.grid[x][y] = 0            
        self.agentPosition = state        
        x, y = self.getAgentRowAndColumn() 
        self.grid[x][y] = 1  

    def offGridMove(self, newState, oldState):
        # if we move into a row not in the grid
        if newState not in self.stateSpacePlus:
            return True
        # if we're trying to wrap around to next row
        elif oldState % self.m == 0 and newState  % self.m == self.m - 1:
            return True
        elif oldState % self.m == self.m - 1 and newState % self.m == 0:
            return True
        else:
            return False        
    def step(self, action):        
        resultingState = self.agentPosition + self.actionSpace[action]
        if not self.offGridMove(resultingState, self.agentPosition):
            self.setState(resultingState)
            return resultingState, -1, self.isTerminalState(resultingState), None
        else:
            return self.agentPosition, -1, self.isTerminalState(self.agentPosition), None
    
    def isTerminalState(self, state):
        return state in self.stateSpacePlus and state not in self.stateSpace      

    def initP(self):
        """ construct state transition probabilities for
        use in value function. P(s', r|s, a) is a dictionary
        with keys corresponding to the functional arguments.
        values are either 1 or 0.
        Translations that take agent off grid leave the state unchanged.
        (s', r|s, a)
        (1, -1|1, 'up') = 1
        (1, -1|2, 'left') = 1
        (1, -1|3, 'left') = 0        
        """
        P = {}
        for state in self.stateSpace:
            for action in self.actionSpace:
                resultingState = state + self.actionSpace[action]
                key = (state, -1, state, action) if self.offGridMove(resultingState, state) \
                                                 else (resultingState, -1, state, action)
                P[key] = 1
        return P

    def render(self):
        print('------------------------------------------')
        for row in self.grid:
            for col in row:
                if col == 0:
                    print('-', end='\t')
                elif col == 1:
                    print('X', end='\t')
            print('\n')
        print('------------------------------------------')

## Del 1 - Utvärdera policyn

In [3]:
def evaluatePolicy(grid, V, policy, GAMMA, THETA):
    # policy evaluation for the random choice in gridworld
    converged = False
    iterations = 0 # <---
    while not converged:
        DELTA = 0
        for state in grid.stateSpace:
            oldV = V[state]
            total = 0
            weight = 1 / len(policy[state])           
            for action in policy[state]:
                grid.setState(state)
                newState, reward, _, _ = grid.step(action)
                key = (newState, reward, state, action)
                total += weight*grid.p[key]*(reward+GAMMA*V[newState])
            V[state] = total
            DELTA = max(DELTA, np.abs(oldV-V[state]))
            converged = True if DELTA < THETA else False
        iterations = iterations + 1 # <---
    print('iterations: ', iterations) # <----
    return V

In [4]:
def printV(V, grid):
    for idx, row in enumerate(grid.grid):
        for idy, _ in enumerate(row):            
            state = grid.m * idx + idy 
            print('%.2f' % V[state], end='\t')
        print('\n')
    print('--------------------')

In [11]:
def get_policy(m, n, x, y):
    grid = GridWorld(m,n) # Rutnät m x n
    THETA = x # Konvergensvillkor
    GAMMA = y
    
    # initialize V(s)
    V = {}
    for state in grid.stateSpacePlus:        
        V[state] = 0
    
    policy = {}
    for state in grid.stateSpace:
        policy[state] = [key for key in grid.actionSpace.keys()]

    
    V = evaluatePolicy(grid, V, policy, GAMMA, THETA)
    printV(V, grid)   

### Hands-on del 1
- Implementera utvärderingspolicyfunktionen (evaluatePolicy) i en python-notebook.
- Testa koden för 4x4, 8x8 och 10x10 rutnät.
    - Vad händer med antalet iterationer som krävs för att konvergera?
    - Vad händer med tiden?
- Vad händer om gamma är lika med 0,5, 0,9 för 4x4 och 8x8? Summera och  gör slutsatser.

In [33]:
get_policy(4,4,0.9,0.9)

iterations:  4
0.00	-3.07	-4.09	-4.39	

-3.07	-4.12	-4.63	-4.68	

-4.09	-4.63	-4.62	-3.86	

-4.39	-4.68	-3.86	0.00	

--------------------


In [34]:
get_policy(8,8,0.9,0.9)

iterations:  5
0.00	-3.54	-4.77	-5.21	-5.36	-5.42	-5.44	-5.44	

-3.54	-4.71	-5.38	-5.69	-5.82	-5.87	-5.89	-5.90	

-4.77	-5.38	-5.77	-5.97	-6.07	-6.11	-6.12	-6.13	

-5.21	-5.69	-5.97	-6.12	-6.19	-6.22	-6.24	-6.24	

-5.36	-5.82	-6.07	-6.19	-6.25	-6.28	-6.27	-6.23	

-5.42	-5.87	-6.11	-6.22	-6.28	-6.27	-6.15	-5.90	

-5.44	-5.89	-6.12	-6.24	-6.27	-6.15	-5.63	-4.53	

-5.44	-5.90	-6.13	-6.24	-6.23	-5.90	-4.53	0.00	

--------------------


In [35]:
get_policy(10,10,0.9,0.9)

iterations:  5
0.00	-3.54	-4.77	-5.21	-5.36	-5.42	-5.44	-5.44	-5.44	-5.45	

-3.54	-4.71	-5.38	-5.69	-5.82	-5.87	-5.89	-5.90	-5.90	-5.90	

-4.77	-5.38	-5.77	-5.97	-6.07	-6.11	-6.12	-6.13	-6.13	-6.13	

-5.21	-5.69	-5.97	-6.12	-6.19	-6.22	-6.24	-6.24	-6.24	-6.25	

-5.36	-5.82	-6.07	-6.19	-6.25	-6.28	-6.29	-6.29	-6.29	-6.30	

-5.42	-5.87	-6.11	-6.22	-6.28	-6.30	-6.31	-6.32	-6.32	-6.31	

-5.44	-5.89	-6.12	-6.24	-6.29	-6.31	-6.32	-6.32	-6.31	-6.26	

-5.44	-5.90	-6.13	-6.24	-6.29	-6.32	-6.32	-6.30	-6.17	-5.92	

-5.44	-5.90	-6.13	-6.24	-6.29	-6.32	-6.31	-6.17	-5.64	-4.53	

-5.45	-5.90	-6.13	-6.25	-6.30	-6.31	-6.26	-5.92	-4.53	0.00	

--------------------


- Den output du har gett ser ut att vara en utskrift av en värdefunktionsuppskattning för varje tillstånd i gridvärlden efter ett visst antal iterationer. Varje värde representerar den förväntade ackumulerade belöningen från det aktuella tillståndet, där negativa värden indikerar att det är bättre att undvika det tillståndet.

- tiden har samma 0 sekunder.



In [37]:
get_policy(8,8,0.9,0.5)

iterations:  2
0.00	-1.42	-1.62	-1.65	-1.66	-1.66	-1.66	-1.66	

-1.42	-1.68	-1.74	-1.75	-1.76	-1.76	-1.76	-1.76	

-1.62	-1.74	-1.77	-1.77	-1.77	-1.77	-1.77	-1.77	

-1.65	-1.75	-1.77	-1.78	-1.78	-1.78	-1.78	-1.78	

-1.66	-1.76	-1.77	-1.78	-1.78	-1.78	-1.78	-1.78	

-1.66	-1.76	-1.77	-1.78	-1.78	-1.78	-1.78	-1.78	

-1.66	-1.76	-1.77	-1.78	-1.78	-1.78	-1.78	-1.61	

-1.66	-1.76	-1.77	-1.78	-1.78	-1.78	-1.61	0.00	

--------------------


In [36]:
get_policy(4,4,0.9,0.5)

iterations:  2
0.00	-1.42	-1.62	-1.65	

-1.42	-1.68	-1.74	-1.75	

-1.62	-1.74	-1.77	-1.61	

-1.65	-1.75	-1.61	0.00	

--------------------


Det verkar som att förändringen av gamma-värdet har en påtaglig effekt på resultaten. Låt oss summera och dra några slutsatser baserat på de två olika gamma-värdena för 4x4-miljön:

För gamma = 0.9:

    Antal iterationer: 4
    Värdefunktionen i varje tillstånd är ganska negativ, vilket indikerar att algoritmen värderar att undvika dessa tillstånd.
    De negativa värdena är mer uttalade, och det finns en tydlig indikation på att undvika vissa tillstånd.

För gamma = 0.5:

    Antal iterationer: 2
    Värdefunktionen är mindre negativ jämfört med gamma = 0.9. De negativa värdena är mindre uttalade.
    Algoritmen verkar vara mindre "straffande" och ger mindre negativa värden för tillstånden.

Slutsatser:

    Påverkan av Gamma: Gamma styr vikten av framtida belöningar i algoritmen. En lägre gamma (0.5) gör att agensen i högre grad fokuserar på kortsiktiga belöningar, medan en högre gamma (0.9) ger större vikt åt långsiktiga belöningar.

    Effekten på Värdefunktionen: För högt gamma (0.9) straffar algoritmen tydligt vissa tillstånd, vilket indikerar att den värderar att undvika dem mer. För lågt gamma (0.5) är straffet mindre uttalat, vilket gör att algoritmen är mindre benägen att undvika vissa tillstånd.

    Konvergenstid: Förändringen av gamma påverkar även antalet iterationer som krävs för konvergens. En lägre gamma leder till snabbare konvergens (2 iterationer) jämfört med en högre gamma (4 iterationer).

Valet av gamma är en viktig aspekt i förstärkningsinlärning och påverkar hur mycket agensen tar hänsyn till framtida belöningar. Det är ofta en avvägning mellan omedelbara och långsiktiga belöningar beroende på det specifika problemet och önskad inlärningsbeteende.

Och dra några slutsatser baserat på de två olika gamma-värdena för 8x8-miljön:
För gamma = 0.9:

    Antal iterationer: 5
    Värdefunktionen är generellt sett mer negativ jämfört med gamma = 0.5. Algoritmen tenderar att straffa tillstånd hårdare.
    De negativa värdena ökar gradvis mot övre vänstra och nedre högra hörnen, vilket indikerar att agensen undviker dessa områden.

För gamma = 0.5:

    Antal iterationer: 2
    Värdefunktionen är mindre negativ jämfört med gamma = 0.9. De negativa värdena är mindre uttalade.
    Algoritmen verkar vara mindre "straffande" och ger mindre negativa värden för tillstånden.

Slutsatser:

    Påverkan av Gamma: Som tidigare diskuterat styr gamma vikten av framtida belöningar. I detta fall har ett lägre gamma (0.5) resulterat i mindre negativa värden och snabbare konvergens jämfört med ett högre gamma (0.9).

    Effekten på Värdefunktionen: För högt gamma (0.9) straffar algoritmen tydligt tillstånden hårdare, medan ett lägre gamma (0.5) ger mindre negativa värden.

    Konvergenstid: Förändringen av gamma påverkar också antalet iterationer som krävs för konvergens. Ett lägre gamma leder till snabbare konvergens (2 iterationer) jämfört med ett högre gamma (5 iterationer).

Sammanfattningsvis påverkar valet av gamma hur mycket agensen tar hänsyn till framtida belöningar och hur snabbt algoritmen konvergerar till en stabil uppskattning av den optimala värdefunktionen. Valet av gamma är en viktig faktor och beror på egenskaperna hos den specifika förstärkta inlärningsuppgiften och önskat beteende hos agensen.

## Del 2 - Förbättra policyn

In [6]:
def improvePolicy(grid, V, policy, GAMMA):
    stable = True
    newPolicy = {}
    for state in grid.stateSpace:       
        oldActions = policy[state]                
        value = []
        newAction = []
        for action in policy[state]:
            grid.setState(state)
            weight = 1 / len(policy[state])
            newState, reward, _, _ = grid.step(action)
            key = (newState, reward, state, action)
            value.append(np.round(weight*grid.p[key]*(reward+GAMMA*V[newState]), 2))
            newAction.append(action)
        value = np.array(value)        
        best = np.where(value == value.max())[0]        
        bestActions = [newAction[item] for item in best] 
        newPolicy[state] = bestActions

        if oldActions != bestActions:
            stable = False
        
    return stable, newPolicy

In [7]:
def get_policy_iteration(m, n, x, y):

    grid = GridWorld(m,n)
    THETA = x
    GAMMA = y
    
    # initialize V(s)

    V = {}
    for state in grid.stateSpacePlus:        
        V[state] = 0

    policy = {}
    for state in grid.stateSpace:
        policy[state] = [key for key in grid.actionSpace.keys()]

    # main loop for policy improvement
    stable = False
    while not stable:
        V = evaluatePolicy(grid, V, policy, GAMMA, THETA)
        stable, policy = improvePolicy(grid, V, policy, GAMMA)       
        V = evaluatePolicy(grid, V, policy, GAMMA, THETA)

        printV(V, grid) 
        time.sleep(2)
        clear_output(wait=True)

    printV(V,grid)

    for state in policy:
        print(state,policy[state])

### Hands-on del 2.

- Implementera den förbättrade policyfunktionen (improvePolicy).
- Testa koden för 4x4, 8x8 och 10x10 rutnät.
    - Vad händer med antalet iterationer som krävs för att konvergera?
    - Vad händer med tiden?
- Hur olika är policyerna?

In [40]:
get_policy_iteration(4,4,0.9,0.9)

0.00	-1.00	-1.90	-2.71	

-1.00	-1.90	-2.71	-1.90	

-1.90	-2.71	-1.90	-1.00	

-2.71	-1.90	-1.00	0.00	

--------------------
1 ['left']
2 ['left']
3 ['left']
4 ['up']
5 ['up', 'left']
6 ['up']
7 ['down']
8 ['up']
9 ['left']
10 ['down', 'right']
11 ['down']
12 ['up']
13 ['right']
14 ['right']


In [41]:
get_policy_iteration(8,8,0.9,0.9)

0.00	-1.00	-1.90	-2.71	-3.44	-4.10	-4.69	-5.22	

-1.00	-1.90	-2.71	-3.44	-4.10	-4.69	-5.22	-5.70	

-1.90	-2.71	-3.44	-4.10	-4.69	-5.22	-5.70	-6.13	

-2.71	-3.44	-4.10	-4.69	-5.22	-5.70	-6.13	-6.51	

-3.44	-4.10	-4.69	-5.22	-5.70	-6.13	-3.44	-2.71	

-4.10	-4.69	-5.22	-5.70	-6.13	-3.44	-2.71	-1.90	

-4.69	-5.22	-5.70	-6.13	-3.44	-2.71	-1.90	-1.00	

-5.22	-5.70	-6.13	-6.51	-2.71	-1.90	-1.00	0.00	

--------------------
1 ['left']
2 ['left']
3 ['left']
4 ['left']
5 ['left']
6 ['left']
7 ['left']
8 ['up']
9 ['up', 'left']
10 ['left']
11 ['up']
12 ['up']
13 ['up']
14 ['up']
15 ['up']
16 ['up']
17 ['up']
18 ['up', 'left']
19 ['up']
20 ['up']
21 ['up']
22 ['up']
23 ['up']
24 ['up']
25 ['left']
26 ['left']
27 ['up', 'left']
28 ['up']
29 ['up']
30 ['up']
31 ['up']
32 ['up']
33 ['left']
34 ['left']
35 ['left']
36 ['up', 'left']
37 ['up']
38 ['down']
39 ['down']
40 ['up']
41 ['left']
42 ['left']
43 ['left']
44 ['left']
45 ['down', 'right']
46 ['down']
47 ['down']
48 ['up']
49 ['left']
50 ['left']
5

In [42]:
get_policy_iteration(10,10,0.9,0.9)

0.00	-1.00	-1.90	-2.71	-3.44	-4.10	-4.69	-5.22	-5.70	-6.13	

-1.00	-1.90	-2.71	-3.44	-4.10	-4.69	-5.22	-5.70	-6.13	-6.51	

-1.90	-2.71	-3.44	-4.10	-4.69	-5.22	-5.70	-6.13	-6.51	-6.86	

-2.71	-3.44	-4.10	-4.69	-5.22	-5.70	-6.13	-6.51	-6.86	-7.18	

-3.44	-4.10	-4.69	-5.22	-5.70	-6.13	-6.51	-6.86	-7.18	-7.46	

-4.10	-4.69	-5.22	-5.70	-6.13	-6.51	-6.86	-4.69	-4.10	-3.44	

-4.69	-5.22	-5.70	-6.13	-6.51	-6.86	-4.69	-4.10	-3.44	-2.71	

-5.22	-5.70	-6.13	-6.51	-6.86	-4.69	-4.10	-3.44	-2.71	-1.90	

-5.70	-6.13	-6.51	-6.86	-7.18	-4.10	-3.44	-2.71	-1.90	-1.00	

-6.13	-6.51	-6.86	-7.18	-7.46	-3.44	-2.71	-1.90	-1.00	0.00	

--------------------
1 ['left']
2 ['left']
3 ['left']
4 ['left']
5 ['left']
6 ['left']
7 ['left']
8 ['left']
9 ['left']
10 ['up']
11 ['up', 'left']
12 ['left']
13 ['up']
14 ['up']
15 ['up']
16 ['up']
17 ['up']
18 ['up']
19 ['up']
20 ['up']
21 ['up']
22 ['up', 'left']
23 ['up']
24 ['up']
25 ['up']
26 ['up']
27 ['up']
28 ['up']
29 ['up']
30 ['up']
31 ['left']
32 ['left']
33 ['up', 

    Vad händer med antalet iterationer som krävs för att konvergera?
       - Ser ut som att det börjar högt numerärt, sjunker sedan och försvinner efter på både 4x4, 8x8 och 10x10.
    Vad händer med tiden?
       - Den stiger från 0, sedan blev det 4 och slutligen 6 s.
    Hur olika är policyerna?
       - Ja, den visar tillstånden för policyerna, men det är fortfarande för många steg tills den når sitt mål.

## Del 3 - Värde iteration

In [8]:
def iterateValues(grid, V, policy, GAMMA, THETA):
    converged = False
    iterations = 0
    while not converged:
        DELTA = 0
        for state in grid.stateSpace:
            oldV = V[state]
            newV = []            
            for action in grid.actionSpace:
                grid.setState(state)
                newState, reward, _, _ = grid.step(action)
                key = (newState, reward, state, action) 
                newV.append(grid.p[key]*(reward+GAMMA*V[newState]))                
            newV = np.array(newV)
            bestV = np.where(newV == newV.max())[0]
            bestState = np.random.choice(bestV)
            V[state] = newV[bestState]
            DELTA = max(DELTA, np.abs(oldV-V[state]))
            converged = True if DELTA < THETA else False
        iterations = iterations + 1
    print('iterations: ', iterations)

    for state in grid.stateSpace:
        newValues = []
        actions = []
        for action in grid.actionSpace:
            grid.setState(state)
            newState, reward, _, _ = grid.step(action)
            key = (newState, reward, state, action)
            newValues.append(grid.p[key]*(reward+GAMMA*V[newState]))
            actions.append(action)
        newValues = np.array(newValues)
        bestActionIDX = np.where(newValues == newValues.max())[0]
        bestActions = actions[np.random.choice(bestActionIDX)]
        bestActions = actions[bestActionIDX[0]]
        policy[state] = bestActions

    return V, policy

In [9]:
def get_value_iteration(m, n, x, y):

    grid = GridWorld(m,n)
    THETA = x
    GAMMA = y

    # initialize V(s)
    V = {}
    for state in grid.stateSpacePlus:        
        V[state] = 0

    policy = {}
    for state in grid.stateSpace:
        policy[state] = [key for key in grid.actionSpace.keys()]

    for i in range(2):
        V, policy = iterateValues(grid, V, policy, GAMMA, THETA)
        printV(V, grid)   
        time.sleep(2)
        clear_output(wait=True)
    # Final   

    printV(V, grid) 
    print()
    for state in policy:
        print(state, policy[state])

### Hands-on del 3.

- Implementera ännu en förbättrad policyfunktion (iterateValues).
- Testa koden för 4x4, 8x8 och 10x10 rutnät.
    - Vad händer med antalet iterationer som krävs för att konvergera?
    - Vad händer med tiden?
- Hur olika är policyerna för gamma=0.5,0.9 i ett 8x8-rutnät?

In [46]:
get_value_iteration(4,4,0.9,0.9)

0.00	-1.00	-1.90	-2.71	

-1.00	-1.90	-2.71	-1.90	

-1.90	-2.71	-1.90	-1.00	

-2.71	-1.90	-1.00	0.00	

--------------------

1 left
2 left
3 down
4 up
5 up
6 up
7 down
8 up
9 up
10 down
11 down
12 up
13 right
14 right


In [47]:
get_value_iteration(8,8,0.9,0.9)

0.00	-1.00	-1.90	-2.71	-2.71	-2.71	-2.71	-2.71	

-1.00	-1.90	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-1.90	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-1.90	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-1.90	-1.00	

-2.71	-2.71	-2.71	-2.71	-2.71	-1.90	-1.00	0.00	

--------------------

1 left
2 left
3 left
4 up
5 up
6 up
7 up
8 up
9 up
10 up
11 up
12 up
13 up
14 up
15 up
16 up
17 up
18 up
19 up
20 up
21 up
22 up
23 up
24 up
25 up
26 up
27 up
28 up
29 up
30 up
31 up
32 up
33 up
34 up
35 up
36 up
37 up
38 up
39 down
40 up
41 up
42 up
43 up
44 up
45 up
46 down
47 down
48 up
49 up
50 up
51 up
52 up
53 down
54 down
55 down
56 up
57 up
58 up
59 up
60 right
61 right
62 right


In [48]:
get_value_iteration(10,10,0.9,0.9)

0.00	-1.00	-1.90	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-1.00	-1.90	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-1.90	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-1.90	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-1.90	-1.00	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-1.90	-1.00	0.00	

--------------------

1 left
2 left
3 left
4 up
5 up
6 up
7 up
8 up
9 up
10 up
11 up
12 up
13 up
14 up
15 up
16 up
17 up
18 up
19 up
20 up
21 up
22 up
23 up
24 up
25 up
26 up
27 up
28 up
29 up
30 up
31 up
32 up
33 up
34 up
35 up
36 up
37 up
38 up
39 up
40 up
41 up
42 up
43 up
44 up
45 up
46 up
47 up
48 up
49 up
50 up
51 up
52 up
53 up
54 up
55 up
56 up
57 up
58 up
59 up
60 up
61 

    Vad händer med antalet iterationer som krävs för att konvergera?
        Det kräver färre iterationer jämfört med del 2 och del 1, och ger bättre resultat med lägre negativa belöningar per steg.

    Vad händer med tiden?
        Utförandet är snabbare jämfört med tidigare skript.

 Hur olika är policyerna för gamma=0.5,0.9 i ett 8x8-rutnät?
   - Med gamma = 0.9 fick snabbare och bättre policyerna mindre - poäng änn 0.5 som fick större värd.
   - För gamma=0.5 verkar algoritmen föredra att röra sig mot det övre vänstra hörnet och att undvika nedre högra hörnet.
   - För gamma=0.9 tenderar algoritmen att föredra att stanna på plats, med några undantag där 'right' föreslås.
   - Resultaten indikerar att olika gamma-värden påverkar optimala strategier och rörelsemönster. En högre gamma (0.9) verkar leda till en mer  konservativ strategi där agensen är mindre benägen att riskera att förflytta sig till nya platser med högre potentiell belöning.

In [49]:
get_value_iteration(10,10,0.5,0.9)

0.00	-1.00	-1.90	-2.71	-3.44	-4.10	-4.69	-5.22	-5.70	-6.13	

-1.00	-1.90	-2.71	-3.44	-4.10	-4.69	-5.22	-5.70	-6.13	-5.70	

-1.90	-2.71	-3.44	-4.10	-4.69	-5.22	-5.70	-6.13	-5.70	-5.22	

-2.71	-3.44	-4.10	-4.69	-5.22	-5.70	-6.13	-5.70	-5.22	-4.69	

-3.44	-4.10	-4.69	-5.22	-5.70	-6.13	-5.70	-5.22	-4.69	-4.10	

-4.10	-4.69	-5.22	-5.70	-6.13	-5.70	-5.22	-4.69	-4.10	-3.44	

-4.69	-5.22	-5.70	-6.13	-5.70	-5.22	-4.69	-4.10	-3.44	-2.71	

-5.22	-5.70	-6.13	-5.70	-5.22	-4.69	-4.10	-3.44	-2.71	-1.90	

-5.70	-6.13	-5.70	-5.22	-4.69	-4.10	-3.44	-2.71	-1.90	-1.00	

-6.13	-5.70	-5.22	-4.69	-4.10	-3.44	-2.71	-1.90	-1.00	0.00	

--------------------

1 left
2 left
3 left
4 left
5 left
6 left
7 left
8 left
9 down
10 up
11 up
12 up
13 up
14 up
15 up
16 up
17 up
18 up
19 down
20 up
21 up
22 up
23 up
24 up
25 up
26 up
27 up
28 down
29 down
30 up
31 up
32 up
33 up
34 up
35 up
36 up
37 down
38 down
39 down
40 up
41 up
42 up
43 up
44 up
45 up
46 down
47 down
48 down
49 down
50 up
51 up
52 up
53 up
54 up
55 down

In [51]:
get_value_iteration(10,10,0.9,0.9)

0.00	-1.00	-1.90	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-1.00	-1.90	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-1.90	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-1.90	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-1.90	-1.00	

-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-2.71	-1.90	-1.00	0.00	

--------------------

1 left
2 left
3 left
4 up
5 up
6 up
7 up
8 up
9 up
10 up
11 up
12 up
13 up
14 up
15 up
16 up
17 up
18 up
19 up
20 up
21 up
22 up
23 up
24 up
25 up
26 up
27 up
28 up
29 up
30 up
31 up
32 up
33 up
34 up
35 up
36 up
37 up
38 up
39 up
40 up
41 up
42 up
43 up
44 up
45 up
46 up
47 up
48 up
49 up
50 up
51 up
52 up
53 up
54 up
55 up
56 up
57 up
58 up
59 up
60 up
61 